# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd

In [ ]:
url= 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df_mark= pd.read_csv(url)

In [ ]:
from info_cleaning import quick_info_dfs
print(quick_info_dfs(df_mark))

In [ ]:
df_mark['State'].fillna('unknown')

In [ ]:
from info_cleaning import standartize_columns

df_mark = standartize_columns(df_mark)
df_mark.columns

In [ ]:
df_mark['months_since_policy_inception']

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [ ]:
# make total_claim_amount is numeric
df_mark['total_claim_amount'] = pd.to_numeric(df_mark['total_claim_amount'], errors='coerce')

# Filter the DataFrame
filtered_df = df_mark[(df_mark['total_claim_amount'] > 1000) & (df_mark['response'] == 'Yes')]

# Show result
filtered_df.head()                   

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [ ]:
response_y=df_mark[df_mark['response'] == 'Yes']
summary = response_y.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2).reset_index()
summary

Personal Auto policyholders — especially males — have the highest average total claim amounts, making them a strong target for marketing.

The lack of wide variance between genders suggests that policy type is more useful for segmentation than gender alone.

Since all averages are above $400, responders in general are valuable customers, even if no extreme highs or lows appear.


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:
#customer to each state
state_counts = df_mark.groupby("state")['customer'].count().sort_values(ascending=False).reset_index()
state_counts.columns = ['state', 'customer_count']
state_counts
#Filtered by customer size is bigger than 500
filtered_state= state_counts[state_counts['customer_count'] > 500]
filtered_state

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
df_mark.groupby(['education','gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)

Higher education ≠ higher customer lifetime value:

Contrary to expectations, Doctoral-level customers have the lowest max and median CLV.

This trend appears across both genders, suggesting lower product fit, loyalty, or engagement.

High School or Below segment is strongest:

Especially males, who show both highest max and highest median CLV.

Females in this group also perform well, making them a highly valuable segment.

Gender Differences Are Modest:

Across all education levels, the difference in median CLV between genders is typically under $500, though males slightly outperform females overall.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [ ]:
df_mark['effective_to_date'] = pd.to_datetime(df_mark['effective_to_date'], format='%m/%d/%Y', errors='coerce')

df_mark['month'] = df_mark['effective_to_date'].dt.strftime('%B')

In [ ]:
# policies_sold_by_state_month = df_mark.pivot_table(index=['state'], columns=['month'], values=['total_claim_amount'])
# policies_sold_by_state_month

#Create pivot table: count of policies sold per state and month
pivot_table = pd.pivot_table(
    df_mark,
    index='state',
    columns='month',
    values='number_of_policies',  # Counting number of policies (1 per row)
    aggfunc='count',
    fill_value=0
)

# # Optional: order months chronologically
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
                'July', 'August', 'September', 'October', 'November', 'December']

pivot_table = pivot_table[[month for month in month_order if month in pivot_table.columns]]

pivot_table

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [ ]:
# Step 1: Count number of policies by state
state_policy_counts = df_mark['state'].value_counts().head(3)
top_3_states = state_policy_counts.index.tolist()

# Step 2: Filter for top 3 states
df_top_states = df_mark[df_mark['state'].isin(top_3_states)]

# Step 3: Group by state and month, count policies
top_states_summary = df_top_states.groupby(['state', 'month'])['policy'].count().reset_index(name='policies_sold')


# Sort the final table
top_states_summary = top_states_summary.sort_values(by=['state', 'month'])
top_states_summary

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# Count total customers and responders by channel
channel_summary = df_mark.groupby('sales_channel')['response'].value_counts().unstack().fillna(0)

# Calculate response rate
channel_summary['response_rate'] = (
    channel_summary['Yes'] / (channel_summary['Yes'] + channel_summary['No'])
).round(2)

# Reset index for readability
channel_summary = channel_summary.reset_index()
channel_summary